In [ ]:

# Install necessary libraries (uncomment if needed)
# !pip install tensorflow numpy pandas matplotlib


# Sentiment Analysis

Bu notebook'ta RNN kullanarak hepsiburada.com sitesinde yapılan ürün yorumları üzerinde sentiment analysis yapıyoruz. Sentiment analysis makine öğrenimi içerisinde yaygın bir problemdir. Amacımız pozitif ve negatif yorumları tahmin edebilen bir model geliştirmek. Hepsiburada'dan yaklaşık 250 bin kadar ürün yorumunu ben daha önce topladım. Aşağıdaki diyagramda görüldüğü gibi bir model bir model eğiteceğiz. Öncelikle bilgisayar text'e anlam veremeyeceği için text'i sayılara çevireceğiz. Bu işleme tokenleştirme diyoruz. Ardından modelin her kelimeye anlamlar yükleyebilmesi için elimizdeki her kelimeyi vektörlere çevireceğiz. Bu işleme ise embedding deniyor. Vektöre çevirdiğimiz kelimeleri oluşturacağımız bir RNN modeline besleyerek modeli eğiteceğiz. RNN için GRU kullanacağız. GRU modelimize input olarak kelime vektörlerinden oluşan yorumu verdikten sonra output olarak 0 ve 1 arasında bir sayı elde edeceğiz. Eğer modelimiz yorumun pozitif olduğunu düşünüyorsa output 1'e, negatif olduğunu düşünüyorsa output 0'a yakın olacak.

![Flowchart](diag/Sent-analysis-diag.png)

In [1]:
import numpy as np
import pandas as pd

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Verileri yüklüyoruz.

In [5]:
dataset = pd.read_csv('hepsiburada.csv')

In [6]:
dataset

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
243492,1,fiyatına göre güzel
243493,1,Ürün kullanışlı iş görüyor fazlasıyla eşime al...
243494,1,"Hızlı Kargo, güzel ürün"
243495,1,telefon başarılı hızlı bir cihaz sadece beyaz...


Hedef 'Rating' sütunu. Bu sütunda her yorumun etiketi bulunabilir. 1 pozitif yorum, 0 negatif yorum.

In [7]:
target = dataset['Rating'].values.tolist()
data = dataset['Review'].values.tolist()

Verinin %80'ini eğitim için %20'sini test için ayırıyoruz. Eğitim esnasında modelin test verilerine erişimi olmayacak.

In [8]:
cutoff = int(len(data) * 0.80)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

Örnek yorumlar:

In [9]:
x_train[500]

'ürün sipariş verdim 2 gün içinde elime ulaştı her zaman ki gibi kullanışlı bi ürün daha once de bu mouse dan almıştım.bu yüzden tereddütsüz aldım . alacak olanlara öneririm'

In [10]:
x_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [11]:
y_train[800]

1

### Tokenleştirme
Datasetindeki kelimeleri integer sayılara çeviriyoruz.

In [12]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

In [13]:
tokenizer.fit_on_texts(data)

In [14]:
tokenizer.word_index

{'çok': 1,
 'bir': 2,
 've': 3,
 'ürün': 4,
 'bu': 5,
 'iyi': 6,
 'güzel': 7,
 'için': 8,
 'tavsiye': 9,
 'ederim': 10,
 'daha': 11,
 'ama': 12,
 'da': 13,
 'gayet': 14,
 'hızlı': 15,
 'teşekkürler': 16,
 'aldım': 17,
 'de': 18,
 'ürünü': 19,
 'gibi': 20,
 'yok': 21,
 'uygun': 22,
 'olarak': 23,
 'kaliteli': 24,
 'en': 25,
 '2': 26,
 'kargo': 27,
 'fiyat': 28,
 'elime': 29,
 'kadar': 30,
 'ile': 31,
 'göre': 32,
 'geldi': 33,
 'var': 34,
 'hepsiburada': 35,
 'ben': 36,
 'gerçekten': 37,
 '1': 38,
 'fiyata': 39,
 'gün': 40,
 'sonra': 41,
 'cok': 42,
 'kesinlikle': 43,
 'telefon': 44,
 'biraz': 45,
 'hiç': 46,
 'ulaştı': 47,
 'memnun': 48,
 'hem': 49,
 'değil': 50,
 'kullanışlı': 51,
 '3': 52,
 'mükemmel': 53,
 'oldu': 54,
 'kullanıyorum': 55,
 'önce': 56,
 'sipariş': 57,
 'tek': 58,
 'her': 59,
 'bence': 60,
 'harika': 61,
 'kalitesi': 62,
 'bi': 63,
 'ayrıca': 64,
 '5': 65,
 'teşekkür': 66,
 'fiyatı': 67,
 'olması': 68,
 'ne': 69,
 'herkese': 70,
 'bile': 71,
 'uzun': 72,
 'süper': 73,

In [15]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)

Örnek tokenler ve elimizdeki veri ile ilgili istatistikler:

In [16]:
x_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [17]:
print(x_train_tokens[800])

[19, 341, 52, 194, 30, 54, 7992, 55, 209, 603, 7887, 3, 36, 114, 164, 479, 85, 1, 1682, 782, 30, 2, 380, 326, 3, 1, 81, 132, 562, 9, 10]


In [18]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [19]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [20]:
np.mean(num_tokens)

20.744703220162876

In [21]:
np.max(num_tokens)

295

In [22]:
np.argmax(num_tokens)

21941

In [23]:
x_train[21941]

'Özellikle bu kısma yazıyorum iyice okuyunuz,cihazın hızı çok iyi.Isınma normal boyutlarda.Kamerası elinizi sabit tutarsanız ve gündüz çok net gece ise çok net çekmez.Görüntülü konuşma özelliği yok ancak uygulama ile olur,anten çekimi gayet iyi.Parmak izi okuyucusu gayet hızlı bazen tuşu silmenizi istiyor ve okuyamıyor kirden vs...Ön kamera da tatmin edici,çekim esnasında ekran beyaz ışık vererek flaş görevi görür.Batarya bana 1 hafta gidiyor sık kullanımda ise 2-3 gün gidiyor.Hızlı şarj 1 saatte doluyor , şekilleri ele oturuyor ve şık bir görüntü var.Telefonu aldığım gün gittim ve ilk girdiğim yerden ekran koruyucu ve kılıf buldum.Kulaklık sesi çok net ve yüksek ancak kendi hoparlörü biraz zayıf sesi.Ekrana bakarken açık kalma özelliği yok.Diğer akıllı cihaz özellikleri %90 ı bu cihazda mevcut.Güç tasarrufu 2 ayrı modu var ve çok başarılı çalışıyor.4gb ram var genelde yarısı boş kalıyor.Bir de yeni cihazların çoğu titreşimi az ve sesi de az çıkıyor.Bu cihaz da içine dahil...Bu cihazla

Sinir ağları veri olarak sabit boyutta input alır. Elimizdeki yorumlar farklı kelime sayılarından oluştuğu için yorumların hepsini sabit bir boyuta getirmemiz gerekiyor. Bu işleme padding denir. Eğer bir yorumda belirlediğimiz sayıdaki tokenden daha az token bulunuyorsa yoruma 0 ekleyerek istediğimiz boyuta getiriyoruz. Eğer yorum belirlediğimiz sayıdan uzunsa yorumun bir kısmını kesiyoruz. Aşağıdaki formül ile token sayısı hesaplıyoruz.

In [24]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

59

In [25]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9597982726686571

In [26]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)

In [27]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [28]:
x_train_pad.shape

(194797, 59)

In [29]:
x_test_pad.shape

(48700, 59)

Padding sonrası örnek yorumlar.

In [30]:
np.array(x_train_tokens[800])

array([  19,  341,   52,  194,   30,   54, 7992,   55,  209,  603, 7887,
          3,   36,  114,  164,  479,   85,    1, 1682,  782,   30,    2,
        380,  326,    3,    1,   81,  132,  562,    9,   10])

In [31]:
x_train_pad[800]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,   19,  341,   52,  194,   30,
         54, 7992,   55,  209,  603, 7887,    3,   36,  114,  164,  479,
         85,    1, 1682,  782,   30,    2,  380,  326,    3,    1,   81,
        132,  562,    9,   10], dtype=int32)

Elimizdeki tokenleri tekrardan text'e çevirmek için kodlar.

In [32]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [33]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [34]:
x_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [35]:
tokens_to_string(x_train_tokens[800])

'ürünü alalı 3 hafta kadar oldu aralıksız kullanıyorum sürekli açık durur ve ben günde yaklaşık 12 saat çok kullanırım şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim'

### Sinir ağı
Keras kullanarak sequential bir model oluşturuyoruz.

In [36]:
model = Sequential()

Modelin embedding kısmında elimizdeki integer tokenleri boyutu 50 olan vektörlere çeviriyoruz. Bu kelime vektörlerini GRU layer'larına input olarak vereceğiz.

In [37]:
embedding_size = 50

In [38]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3 layer'lı GRU modeli oluşturuyoruz. Bir GRU'dan çıkan output diğer GRU'ya input olarak verileceği için ilk iki GRU'da return_sequences=True dememiz gerekir. Üçüncü GRU'u ise bir dense layer'a bağlıyoruz. Bu noktada return_sequences=False olmalı. Default olarak böyle zaten. Son dense layer'da aktivasyon fonksiyonu olarak sigmoid kullanıyoruz. Sigmoid sayıları 0 ve 1 arasına sıkıştırır. Çıkan output 1'e yakınsa model yorumun pozitif olduğunu düşünüyor demektir. 0'a yakınsa negatif.

In [39]:
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

In [46]:
import tensorflow as tf

In [47]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

Oluşturduğumuz modelin özeti:

In [49]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_layer (Embedding)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Bu noktada modeli eğitiyoruz. Input olarak yorumların tokenlerini yani x_train_pad'i ve bunların etiketlerini yani y_train'i veriyoruz. Toplamda 256 batch_size ile 5 epoch kadar modeli eğitiyoruz.

In [51]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [52]:
model.fit(x_train_pad, y_train, validation_split=0.1, epochs=5, batch_size=256)

Epoch 1/5
685/685 ━━━━━━━━━━━━━━━━━━━━ 16s 14ms/step - accuracy: 0.9421 - loss: 0.2557 - val_accuracy: 0.9690 - val_loss: 0.0949
Epoch 2/5
685/685 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - accuracy: 0.9709 - loss: 0.0920 - val_accuracy: 0.9722 - val_loss: 0.0816
Epoch 3/5
685/685 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.9783 - loss: 0.0712 - val_accuracy: 0.9725 - val_loss: 0.0845
Epoch 4/5
685/685 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9843 - loss: 0.0560 - val_accuracy: 0.9705 - val_loss: 0.0903
Epoch 5/5
685/685 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9894 - loss: 0.0417 - val_accuracy: 0.9703 - val_loss: 0.0978


Eğitilen modeli test setinde test ediyoruz.

In [53]:
result = model.evaluate(x_test_pad, y_test)

1522/1522 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9447 - loss: 0.1746


Sonuç olarak model test setinde 0.9447 accuracy oranına erişiyor.

In [54]:
result[1]

0.9527104496955872

Ürünün yanlış bildiği yorumlara örnek:

In [55]:
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


In [56]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [57]:
cls_true = np.array(y_test[0:1000])

In [58]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [59]:
len(incorrect)

52

In [60]:
idx = incorrect[0]
idx

5

In [61]:
text = x_test[idx]
text

'Ürün güzel ama teslimat çok geç yapıldı.'

In [62]:
y_pred[idx]

0.7350859

In [63]:
cls_true[idx]

0

Son olarak datasetinde olmayan kendi yazdığım 10 tane yorumu örnek olması için modelden geçiriyorum.

In [64]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [65]:
tokens = tokenizer.texts_to_sequences(texts)

In [66]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

(10, 59)

Model kendi yazdığım yorumların hepsini doğru olarak bilebildi.

In [67]:
model.predict(tokens_pad)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


array([[0.99716634],
       [0.9961158 ],
       [0.5813255 ],
       [0.99694246],
       [0.05501722],
       [0.16238628],
       [0.9952859 ],
       [0.02061399],
       [0.99628973],
       [0.2720952 ]], dtype=float32)

In [ ]:
from google.colab import files
files.download('model.h5')  # model.h5 dosyasını indirmek için.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>